## Wealth Distributions

### Constant Savings Rate, Impact of Volatility

In [1]:
import numpy as np
from numba import jit, njit, prange
import matplotlib.pyplot as plt


### Lorenz Curves and Gini Coefficient



A function to produce Lorenz curves

In [2]:
@njit
def lorenz(y):
    n = len(y)
    y = np.sort(y)
    s = np.zeros(n+1)
    s[1:] = np.cumsum(y)
    f_vals = np.zeros(n+1)
    l_vals = np.zeros(n+1)
    for i in range(1, n+1):
        f_vals[i] = i/n
        l_vals[i] = s[i] / s[n]
    return f_vals, l_vals

A function to calculate the Gini coefficient

In [3]:
@njit(parallel=True)
def gini(y):
    n = len(y)
    i_sum = np.zeros(n)
    for i in prange(n):
        for j in range(n):
            i_sum[i] += abs(y[i] - y[j])
    return np.sum(i_sum) / (2 * n * np.sum(y))

### Dynamics

The model is

$$ w_{t+1} = (1 + r_{t+1}) s_0 w_t + y_{t+1} $$

where

* $w_t$ is wealth at time $t$ for a given household,
* $r_t$ is the rate of return of financial assets,
* $y_t$ is current non-financial (e.g., labor) income and
* $s_0$ is a savings rate


Letting $\{z_t\}$ be a correlated state process of the form

$$ z_{t+1} = a z_t + b + \sigma_z \epsilon_{t+1} $$

we'll assume that

$$ \ln R_t = \mu_r + c_r \exp(z_t) \xi_t $$

and 

$$ y_t = c_y \exp(z_t) + \exp(\mu_y + \sigma_y \zeta_t) $$

Here $\{ (\epsilon_t, \xi_t, \zeta_t) \}$ is IID and standard normal in $\mathbb R^3$.


In [17]:
class WealthDynamics:
    
    def __init__(self,
                 s_0=0.75,    # savings parameter
                 c_y=1.0,     # labor income parameter
                 μ_y=1.5,     # labor income paraemter
                 σ_y=0.2,     # labor income parameter
                 μ_r=0.0,     # rate of return parameter
                 c_r=0.5,     # rate of return parameter
                 a=0.9,       # z shock parameter
                 b=0.0,       # z shock parameter
                 σ_z=0.1):    # z shock parameter
    
        # Boilerplate
        self.s_0 = s_0
        self.c_y, self.μ_y, self.σ_y = c_y, μ_y, σ_y
        self.c_r, self.μ_r = c_r, μ_r
        self.a, self.b, self.σ_z = a, b, σ_z
        
        # Record stationary moments
        self.z_mean = b / (1 - a)
        self.z_var = σ_z**2 / (1 - a**2)
        exp_z_mean = np.exp(self.z_mean + self.z_var / 2)
        
        # Compute stationary mean for R and Y by MC
        

        
        # Test stability condition, compute estimate of the stationary
        # mean of w_t
        α = self.R_mean * self.s_0
        if α < 1:
            print(f"Generated instance with stability coefficient α = {α}")
            self.w_bar = self.y_mean / (1 - α)
        else:
            raise ValueError(f"Stability condition failed with α = {α}")
            
    def compute_stationary_means(self):
        # Unpack parameters
        params = self.parameters()
        s_0, c_y, μ_y, σ_y, c_r, μ_r, a, b, σ_z = params
                
        @njit
        def csm(        mc_size = 10_000
):
            z_draws = self.z_mean + np.sqrt(self.z_var) * np.random.randn(mc_size)
            ξ_draws = np.random.randn(mc_size)
            R_mean = 0.0
            Y_mean = 0.0
            for z in z_draws:
                for ξ in ξ_draws:
                    R_mean += np.exp(μ_r + c_r ) * z * ξ
                    Y_mean += c_y * np.exp(z) + np.exp(μ_y + σ_y * ξ)
            R_mean = R_mean / (mc_size**2)
            Y_mean = Y_mean / (mc_size**2)

    def parameters(self):
        """
        Collect and return parameters.
        """
        parameters = (self.s_0, 
                      self.c_y, self.μ_y, self.σ_y,
                      self.c_r, self.μ_r,
                      self.a, self.b, self.σ_z)
        return parameters
        
    def update_function_builder(self):
        """
        A function factory that returns a function with one task:
        update wealth from w_t to w_{t+1}.
        """
        # Unpack parameters
        params = self.parameters()
        s_0, c_y, μ_y, σ_y, c_r, μ_r, a, b, σ_z = params
                
        @njit
        def update_z(z):
            """
            Update exogenous state one period, given current state z.
            """   
            zp = a * z + b + σ_z * np.random.randn()
            return zp
        
        @njit
        def update_w(w, z):
            """
            Update one period, given current wealth w and next period 
            exogenous state zp.
            """   
            y = c_y * np.exp(z) + np.exp(μ_y + σ_y * np.random.randn())   
            R = np.exp(μ_r + c_r * np.exp(z) * np.random.randn())
            wp = R * s_0 * w + y
            return wp
        
        return update_z, update_w
    

Here's a function to simulate wealth of a single household.

In [18]:
@njit
def wealth_time_series(w_0, z_0, g, h, ts_length=10_000):
    """
    Generate a single time series for wealth.
    """
    z = np.empty(ts_length)
    z[0] = z_0
    w = np.empty(ts_length)
    w[0] = w_0
    for t in range(ts_length-1):
        z[t+1] = g(z[t])
        w[t+1] = h(w[t], z[t+1])
    return w

Here's a function to simulate a cross section.

In [19]:
def cross_section(wd, w_distribution, z_0, shift_length=500):
    """
    Takes a current distribution of wealth values as w_distribution
    and updates each w_t in w_distribution to w_{t+j}, where 
    j = shift_length.  
    
    The parameter wd is an instance of WealthDynamics
    
    The value z_0 is the initial condition of the aggregate state.
    
    Returns the new distribution.
    """
    n = len(w_distribution)

    g, h = wd.update_function_builder()
    z = z_time_series(z_0, g, ts_length=shift_length)

    @njit(parallel=True)
    def simulate_forward(out_distribution):
        
        # Update each household
        for i in prange(n):
            w = w_distribution[i]
            for t in range(shift_length-1):
                w = h(w, z[t+1])
            out_distribution[i] = w
        return out_distribution

    return simulate_forward(np.empty(n))

### Experiments

Let's look at the wealth dynamics of an individual household.  

In [20]:
wd = WealthDynamics()

g, h = wd.update_function_builder()
z = z_time_series(0.0, g, ts_length=100_000)
w = wealth_time_series(wd.y_mean, z, h)

fig, ax = plt.subplots()
ax.plot(w)
plt.show()

Generated instance with stability coefficient α = 7.400739280103743e-06


AttributeError: 'WealthDynamics' object has no attribute 'y_mean'

### Application II: Inequality Measures

Let's look at how inequality varies with returns on financial assets.

First we need a function to simulate a cross section of households forward in time.

The next function generates a cross section and then computes the Lorenz curve and Gini coefficient.

In [ ]:
def generate_lorenz_and_gini(wd, num_households=250_000, T=500):
    """
    Generate the Lorenz curve data and gini coefficient corresponding to a 
    WealthDynamics mode by simulating num_households forward to time T.
    """
    ψ_0 = np.ones(num_households) * wd.y_mean
    z_0 = wd.z_mean

    ψ_star = cross_section(wd, ψ_0, z_0, shift_length=T)
    return gini(ψ_star), lorenz(ψ_star)

Here's the Lorenz curves as a function of $\mu_r$, a parameter that shifts up mean returns.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
μ_r_vals = np.linspace(0.0, 0.05, 3)
gini_vals = []

for μ_r in μ_r_vals:
    wd = WealthDynamics(μ_r=μ_r)
    gv, (f_vals, l_vals) = generate_lorenz_and_gini(wd)
    ax.plot(f_vals, l_vals, label=f'$\psi^*$ at $\mu_r = {μ_r:0.2}$')
    gini_vals.append(gv)
    
ax.plot(f_vals, f_vals, label='equality')
ax.legend()
plt.show()

Here's the Gini coefficient.  Look how inequality.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))    
ax.plot(μ_r_vals, gini_vals, label='gini coefficient')
ax.set_xlabel("volatility term in returns")
ax.legend()
plt.show()